In [1]:
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
import time
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import clear_output
from joblib import Parallel, delayed
from tqdm import tqdm
import time
import xlwings

In [2]:
timerframe_list = [
    "day",
    "minute",
    "3minute",
    "5minute",
    "10minute",
    "15minute",
    "30minute",
    "60minute",
]

# instument in nse
inst = pd.DataFrame(kite.instruments("NSE"))
inst_filter = inst.query('(name != "")').copy()
inst_filter.rename(columns={"tradingsymbol": "Symbol"}, inplace=True)
# inst_filter.query("Symbol. 'NIFTY'")
# inst_filter[inst_filter.Symbol.str.contains('HDFCBANK')]

nifty_500_df = pd.read_csv("../data/ind_nifty500list.csv")
nifty_500_df = nifty_500_df.merge(inst_filter, how="left", on="Symbol").dropna(
    subset=["instrument_token"]
)
inst_dict = dict(zip(nifty_500_df.Symbol, nifty_500_df.instrument_token.astype(int)))


In [3]:
def get_day_pivot_added(df_day_fetch, df_min_5):
    df_day_fetch_temp = df_day_fetch.copy()

    # calculating pivot
    df_day_fetch["pivot"] = (
        df_day_fetch["High"] + df_day_fetch["Low"] + df_day_fetch["Close"]
    ) / 3
    df_day_fetch["bottom_pivot"] = (df_day_fetch["High"] + df_day_fetch["Low"]) / 2
    df_day_fetch["top_pivot"] = (
        df_day_fetch["pivot"] - df_day_fetch["bottom_pivot"]
    ) + df_day_fetch["pivot"]
    df_day_fetch = df_day_fetch.shift(1)

    # adding preiouse day price
    df_day_fetch.rename(
        columns={
            "Close": "prev_close",
            "High": "prev_high",
            "Low": "prev_low",
            "Open": "prev_open",
        },
        inplace=True,
    )
    df_day_fetch = df_day_fetch_temp.merge(
        df_day_fetch, left_index=True, right_index=True
    )

    # merging pivot with 5 min candle
    df_min_5["date_only"] = pd.to_datetime(df_min_5.index.date)
    df_day_fetch["date_only"] = pd.to_datetime(df_day_fetch.index.date)
    df_day_fetch.rename(
        columns={
            "Close": "day_close",
            "High": "day_high",
            "Low": "day_low",
            "Open": "day_open",
        },
        inplace=True,
    )
    df_min_5.reset_index(inplace=True)
    df_min_5 = df_min_5.merge(df_day_fetch, how="left", on="date_only")
    df_min_5.set_index("date", inplace=True)
    df_min_5.dropna(inplace=True)
    df_min_5.drop(["Volume_x", "Volume_y"], inplace=True, axis=1)

    return df_min_5


In [12]:
start_dt = "2023-09-01"
end_dt = "2023-09-05"
time_frame = "5minute"
instument = 256265
df_pivot_data = pd.DataFrame()

rename_cols = {
    "open": "Open",
    "high": "High",
    "low": "Low",
    "close": "Close",
    "volume": "Volume",
}
for sy, itoken in tqdm(inst_dict.items()):
    df_5min = pd.DataFrame(
        kite.historical_data(
            instrument_token= int(itoken),
            from_date=start_dt,
            to_date=end_dt,
            interval=time_frame,
        )
    ).assign(Symbol=sy)
    df_day = pd.DataFrame(
        kite.historical_data(
            instrument_token= int(itoken),
            from_date=start_dt,
            to_date=end_dt,
            interval="day",
        )
    ).assign(Symbol=sy)
    df_5min = df_5min.rename(columns=rename_cols).set_index("date")
    df_day = df_day.rename(columns=rename_cols).set_index("date")
    df_pivot_data = df_pivot_data.append(get_day_pivot_added(df_day, df_5min))
    # break


100%|██████████| 490/490 [01:22<00:00,  5.97it/s]


In [13]:
df_pivot_data.to_parquet("../data/pivot_data.parquet")

In [53]:
df_pivot_data = pd.read_parquet("../data/pivot_data.parquet")

In [54]:
df_pivot_data['norrow_cpr'] = abs(df_pivot_data['bottom_pivot'] - df_pivot_data['top_pivot'])/ df_pivot_data['bottom_pivot'] * 100

In [55]:
df_pivot_data.sort_values('norrow_cpr')

,Open,High,Low,Close,Volume,Symbol,date_only,day_open,day_high,day_low,...,Symbol_x,prev_open,prev_high,prev_low,prev_close,Symbol_y,pivot,bottom_pivot,top_pivot,norrow_cpr
date,,,,,,,,,,,,,,,,,,,,,
2023-09-04 13:30:00+05:30,14.95,15.05,14.95,15.00,111396,INFIBEAM,2023-09-04,14.90,15.35,14.80,...,INFIBEAM,14.60,15.05,14.45,14.75,INFIBEAM,14.75,14.75,14.75,0.000000
2023-09-04 13:15:00+05:30,15.00,15.00,14.95,14.95,85268,INFIBEAM,2023-09-04,14.90,15.35,14.80,...,INFIBEAM,14.60,15.05,14.45,14.75,INFIBEAM,14.75,14.75,14.75,0.000000
2023-09-04 13:20:00+05:30,14.95,15.00,14.95,14.95,29073,INFIBEAM,2023-09-04,14.90,15.35,14.80,...,INFIBEAM,14.60,15.05,14.45,14.75,INFIBEAM,14.75,14.75,14.75,0.000000
2023-09-04 13:25:00+05:30,15.00,15.00,14.95,14.95,26675,INFIBEAM,2023-09-04,14.90,15.35,14.80,...,INFIBEAM,14.60,15.05,14.45,14.75,INFIBEAM,14.75,14.75,14.75,0.000000
2023-09-04 15:45:00+05:30,15.35,15.35,15.35,15.35,155,BCG,2023-09-04,15.35,15.35,15.35,...,BCG,16.15,16.15,16.15,16.15,BCG,16.15,16.15,16.15,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-04 09:20:00+05:30,137.25,138.15,136.45,136.50,3581769,BHEL,2023-09-04,137.25,140.60,134.65,...,BHEL,122.00,137.10,121.40,136.15,BHEL,131.55,129.25,133.85,3.558994
2023-09-04 09:25:00+05:30,136.50,137.10,136.05,136.45,2205518,BHEL,2023-09-04,137.25,140.60,134.65,...,BHEL,122.00,137.10,121.40,136.15,BHEL,131.55,129.25,133.85,3.558994
2023-09-04 09:30:00+05:30,136.40,137.65,136.15,137.50,2464594,BHEL,2023-09-04,137.25,140.60,134.65,...,BHEL,122.00,137.10,121.40,136.15,BHEL,131.55,129.25,133.85,3.558994


In [57]:
# df_pivot_data.query('Symbol == "ABFRL"')

In [58]:
df_pivot_data = df_pivot_data.query('norrow_cpr <.05')

In [59]:
req_cols = ["Close","pivot","bottom_pivot","top_pivot",'Symbol',"prev_close"]
df_pivot_data = df_pivot_data[req_cols]

In [60]:
df_pivot_data.head()

,Close,pivot,bottom_pivot,top_pivot,Symbol,prev_close
date,,,,,,
2023-09-04 09:15:00+05:30,355.80,352.416667,352.5,352.333333,AWL,352.25
2023-09-04 09:20:00+05:30,355.55,352.416667,352.5,352.333333,AWL,352.25
2023-09-04 09:25:00+05:30,355.85,352.416667,352.5,352.333333,AWL,352.25
2023-09-04 09:30:00+05:30,355.00,352.416667,352.5,352.333333,AWL,352.25
2023-09-04 09:35:00+05:30,354.00,352.416667,352.5,352.333333,AWL,352.25


In [61]:
df_temp= df_pivot_data.reset_index().set_index(["date","Symbol"]).merge(
(df_pivot_data#.between_time("09:15", "09:16")
.apply(
    lambda x: "1" +"_" +str(x["Symbol"])
    if x["Close"] > x["top_pivot"] and x["Close"] > x["bottom_pivot"]
    else "100" +"_" + str(x["Symbol"])
    if x["Close"] < x["top_pivot"] and x["Close"] < x["bottom_pivot"]
    else "0" +"_" +str(x["Symbol"]),
    axis=1,
)
.reset_index()
.rename(columns={0: "temp_flag"})
.assign(pivot_flag=lambda x: x.temp_flag.str.split("_").str[0])
.assign(Symbol=lambda x: x.temp_flag.str.split("_").str[1])
.drop("temp_flag", axis=1)
.astype({'pivot_flag': int})
.set_index(["date","Symbol"]) ), 
how = "left",
left_index=True,
right_index=True,
)
df_temp= df_temp.reset_index()
df_temp.date = pd.to_datetime(df_temp.date)

In [74]:
def up_down_flag(df):
    df = df.set_index('date')
    df = df.assign(up_flag = df.between_time("09:15", "9:21").pivot_flag.sum() == 2)
    df = df.assign(down_flag = df.between_time("09:15", "09:21").pivot_flag.sum() == 200)
    return df
df_temp = df_temp.groupby('Symbol').apply(up_down_flag).drop('Symbol', axis=1).reset_index()

In [75]:
df_temp.head(20)

,Symbol,date,Close,pivot,bottom_pivot,top_pivot,prev_close,pivot_flag,up_flag,down_flag
0,ASTERDM,2023-09-04 09:15:00+05:30,330.50,329.966667,329.975,329.958333,329.95,1,True,False
1,ASTERDM,2023-09-04 09:20:00+05:30,332.30,329.966667,329.975,329.958333,329.95,1,True,False
2,ASTERDM,2023-09-04 09:25:00+05:30,331.75,329.966667,329.975,329.958333,329.95,1,True,False
3,ASTERDM,2023-09-04 09:30:00+05:30,331.00,329.966667,329.975,329.958333,329.95,1,True,False
4,ASTERDM,2023-09-04 09:35:00+05:30,330.80,329.966667,329.975,329.958333,329.95,1,True,False
5,ASTERDM,2023-09-04 09:40:00+05:30,330.60,329.966667,329.975,329.958333,329.95,1,True,False
6,ASTERDM,2023-09-04 09:45:00+05:30,330.45,329.966667,329.975,329.958333,329.95,1,True,False
7,ASTERDM,2023-09-04 09:50:00+05:30,330.45,329.966667,329.975,329.958333,329.95,1,True,False
8,ASTERDM,2023-09-04 09:55:00+05:30,330.75,329.966667,329.975,329.958333,329.95,1,True,False
9,ASTERDM,2023-09-04 10:00:00+05:30,331.20,329.966667,329.975,329.958333,329.95,1,True,False


In [76]:
df_temp.query("up_flag == True and pivot_flag == 100").groupby('Symbol').date.min().reset_index()

,Symbol,date
0,ASTERDM,2023-09-04 13:10:00+05:30
1,ATUL,2023-09-04 09:45:00+05:30
2,AWL,2023-09-04 10:55:00+05:30
3,BEML,2023-09-04 12:00:00+05:30
4,DABUR,2023-09-04 10:00:00+05:30
5,HEROMOTOCO,2023-09-04 12:45:00+05:30
6,JUBLINGREA,2023-09-04 09:30:00+05:30
7,MCDOWELL-N,2023-09-04 09:30:00+05:30
8,MPHASIS,2023-09-04 09:50:00+05:30
9,OBEROIRLTY,2023-09-04 14:55:00+05:30


In [77]:
df_temp.query("up_flag == True and pivot_flag == 100").groupby('Symbol').date.min().reset_index().Symbol.unique().tolist()

['ASTERDM',
 'ATUL',
 'AWL',
 'BEML',
 'DABUR',
 'HEROMOTOCO',
 'JUBLINGREA',
 'MCDOWELL-N',
 'MPHASIS',
 'OBEROIRLTY',
 'RAINBOW',
 'RELIANCE',
 'SYMPHONY']

In [78]:
df_temp.query("down_flag == True").groupby('Symbol').date.min().reset_index()

,Symbol,date
0,BCG,2023-09-04 09:15:00+05:30
1,ESCORTS,2023-09-04 09:15:00+05:30
2,KEC,2023-09-04 09:15:00+05:30


In [ ]:
#write to find compounding return
#  


In [ ]:
# compound intrest forumula
# A = P(1 + r/n)nt
# A = the future value of the investment/loan, including interest
# P = the principal investment amount (the initial deposit or loan amount)
# r = the annual interest rate (decimal)
# n = the number of times that interest is compounded per year
# t = the number of years the money is invested or borrowed for


In [ ]:
# generate sequence in for loop 2,4,8,16,32,64,128,256,512,1024,2048,4096,8192
# 2**i for i in range(1,14)
# 2**i for i in range(1,14)


In [208]:
# for r, c in df_temp.iterrows():
#     if c['up_flag']:
#         print(r, c['Symbol'])
#     if c['down_flag']:
#         print(r, c['Symbol'])

2023-09-01 09:15:00+05:30 nothing


In [96]:
df_temp#.groupby(['Symbol','date_only'])

,Close,pivot,bottom_pivot,top_pivot,Symbol,pivot_flag
date,,,,,,
2023-09-01 09:15:00+05:30,19293.75,19288.55,19305.925,19271.175,3MINDIA,nothing
2023-09-01 09:20:00+05:30,19291.10,19288.55,19305.925,19271.175,3MINDIA,nothing
2023-09-01 09:25:00+05:30,19293.30,19288.55,19305.925,19271.175,3MINDIA,nothing
2023-09-01 09:30:00+05:30,19312.20,19288.55,19305.925,19271.175,3MINDIA,up
2023-09-01 09:35:00+05:30,19312.40,19288.55,19305.925,19271.175,3MINDIA,up
...,...,...,...,...,...,...
2023-09-01 15:05:00+05:30,19442.75,19288.55,19305.925,19271.175,3MINDIA,up
2023-09-01 15:10:00+05:30,19435.15,19288.55,19305.925,19271.175,3MINDIA,up
2023-09-01 15:15:00+05:30,19426.85,19288.55,19305.925,19271.175,3MINDIA,up


In [ ]:
df_15 = pd.DataFrame(
            kite.historical_data(
                instrument_token=int(itoken),
                from_date=start_dt,
                to_date=end_dt,
                interval="15minute",
            )
)
df_hour = pd.DataFrame(
            kite.historical_data(
                instrument_token=int(itoken),
                from_date="2023-08-01",
                to_date=end_dt,
                interval="60minute",
            )
)
df_day = pd.DataFrame(
            kite.historical_data(
                instrument_token=int(itoken),
                from_date="2023-07-01",
                to_date=end_dt,
                interval="day",
            )
)

In [ ]:


start_dt = "2023-08-09"
end_dt = "2023-08-11"
# return n dict pair
# def get_n_dict_pair(n, dict):
#     return {k: dict[k] for k in list(dict.keys())[:n]}

# results = Parallel(n_jobs=2, backend = 'threading')(delayed(get_intraday_rsi_5star)(sy, itoken) for sy, itoken in tqdm(get_n_dict_pair(100, inst_dict).items()))
# df_final = pd.concat(results)
wb = xlwings.Book(r'D:\US\McD-Discount-Portal\mcdds\notebooks\akash_dev\test.xlsx')
sht = wb.sheets['buy_sell']
current_time = 0
while True:
    if current_time%5 == 0: 
        time.sleep(5) 
        df_final = pd.DataFrame()

        for sy, itoken in tqdm(inst_dict.items()):
            try:
                df_final = df_final.append(get_intraday_rsi_5star(sy, itoken,sht))
            except:
                print(sy)
                continue
            # break

        
        df_final = df_final.query('daily_date == @end_dt and close <=2000')
        df_final.to_csv(f'data/test_11_aug_{datetime.now().strftime("%H_%M")}.csv')
        sht.clear_contents()
        sht.range('A1').value = df_final[df_final['buy']].sort_index()[['close','symbol']]
        sht.range('E1').value = df_final[df_final['sell']].sort_index()[['close','symbol']]
        # break
        # time.sleep(60*5)
    break
    current_time = datetime.now().minute